# Chi-cuadrada

### Librerías

In [53]:
import medmnist
from medmnist import INFO
from medmnist.dataset import DermaMNIST

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import chi2
# 
from tqdm import tqdm
import dataset_without_pytorch
from dataset_without_pytorch import get_loader

from sklearn.cluster import KMeans
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### Carga de datos

In [2]:
data_flag = 'dermamnist'
download = True

BATCH_SIZE = 128 #Lote tamanio

info_28 = INFO[data_flag] # Info

# Obtener dinamicamente la clase del conjunto de datos desde un modulo
DataClass_28 = getattr(dataset_without_pytorch, info_28['python_class'])

train_dataset_28 = DataClass_28(split='train', download=download)

# Encapsular los datos en un DataLoader para hacer lotes
train_loader_28 = get_loader(dataset=train_dataset_28, batch_size=BATCH_SIZE)

images = train_dataset_28.imgs
labels = train_dataset_28.labels

images_class_5 = [image for image, label in zip(images, labels) if label == 5]

flattened_images = [image.flatten() for image in images_class_5]

# Aplicar K-means con 1000 clusters
kmeans = KMeans(n_clusters=1000, random_state=42)
kmeans.fit(flattened_images)

# Obtener los centros de los clusters
cluster_centers = kmeans.cluster_centers_

# Para cada centroide, encontrar la imagen más cercana en el conjunto original
selected_images = []
for center in cluster_centers:
    # Calcular la distancia entre el centroide y todas las imágenes
    distances = np.linalg.norm(flattened_images - center, axis=1)
    # Encontrar el índice de la imagen más cercana
    closest_image_index = np.argmin(distances)
    # Añadir la imagen más cercana a la lista de imágenes seleccionadas
    selected_images.append(images_class_5[closest_image_index])

# Filtrar imágenes y etiquetas de todas las clases excepto la clase 5
images_other_classes = [image for image, label in zip(images, labels) if label != 5]
labels_other_classes = [label for label in labels if label != 5]

# Crear etiquetas para las imágenes seleccionadas de la clase 5
selected_labels = [5] * len(selected_images)

# Combinar las imágenes y etiquetas de las otras clases con las seleccionadas de la clase 5
new_images = images_other_classes + selected_images
new_labels = labels_other_classes + selected_labels

new_labels = [int(label[0]) if isinstance(label, (list, np.ndarray)) else int(label) for label in new_labels]

new_images = np.array(new_images)
new_labels = np.array(new_labels)

# Calcular la distribución de clases
unique, counts = np.unique(new_labels, return_counts=True)
class_distribution = dict(zip(unique, counts))

# Calcular el promedio de imágenes por clase
average_images = int(np.mean(list(class_distribution.values())))

# Crear un generador de datos para rotación
datagen = ImageDataGenerator(rotation_range=40)

# Listas para las imágenes y etiquetas aumentadas
augmented_images = []
augmented_labels = []

# Aumentar imágenes en las clases 3 y 6 hasta alcanzar el promedio
target_classes = [3, 6]
for target_class in target_classes:
    class_images = new_images[new_labels == target_class]
    current_count = class_distribution[target_class]
    images_to_add = average_images - current_count

    if images_to_add > 0:
        for i in range(images_to_add):
            # Seleccionar una imagen aleatoria de la clase
            img = class_images[i % len(class_images)]
            img = np.expand_dims(img, 0)  # Expande dimensiones para el generador

            # Generar una imagen aumentada
            for batch in datagen.flow(img, batch_size=1):
                augmented_images.append(batch[0].astype(np.uint8))  # Agregar imagen aumentada
                augmented_labels.append(target_class)
                break  # Solo necesitamos una imagen por iteración

# Agregar las imágenes aumentadas al conjunto original
new_images = np.concatenate([new_images, np.array(augmented_images)])
new_labels = np.concatenate([new_labels, np.array(augmented_labels)])

# Confirmar la nueva distribución
unique, counts = np.unique(new_labels, return_counts=True)
new_class_distribution = dict(zip(unique, counts))


Using downloaded and verified file: C:\Users\abiab\.medmnist\dermamnist.npz


c:\Users\abiab\AppData\Local\Programs\Python\Python312\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\abiab\AppData\Local\Programs\Python\Python312\Lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


In [3]:
actinic_keratoses_intraepithelial_carcinoma_28 = []
basal_cell_carcinoma_28 = []
benign_keratosis_like_lesions_28 = []
dermatofibroma_28 = []
melanoma_28 = []
melanocytic_nevi_28 = []
vascular_lesions_28 = []


for img,il in zip(new_images, new_labels):
    match il:
        case 0:
            actinic_keratoses_intraepithelial_carcinoma_28.append(img)
        case 1:
            basal_cell_carcinoma_28.append(img)
        case 2:
            benign_keratosis_like_lesions_28.append(img)
        case 3:
            dermatofibroma_28.append(img)
        case 4:
            melanoma_28.append(img)
        case 5:
            melanocytic_nevi_28.append(img)
        case 6:
            vascular_lesions_28.append(img)
            

actinic_keratoses_intraepithelial_carcinoma_28 = np.array(actinic_keratoses_intraepithelial_carcinoma_28)
basal_cell_carcinoma_28 = np.array(basal_cell_carcinoma_28)
benign_keratosis_like_lesions_28 = np.array(benign_keratosis_like_lesions_28)
dermatofibroma_28 = np.array(dermatofibroma_28)
melanoma_28 = np.array(melanoma_28)
melanocytic_nevi_28 = np.array(melanocytic_nevi_28)
vascular_lesions_28 = np.array(vascular_lesions_28)

print("actinic_keratoses_intraepithelial_carcinoma_28 shape:", actinic_keratoses_intraepithelial_carcinoma_28.shape)
print("basal_cell_carcinoma_28 shape:", basal_cell_carcinoma_28.shape)
print("benign_keratosis_like_lesions_28 shape:", benign_keratosis_like_lesions_28.shape)
print("dermatofibroma_28 shape:", dermatofibroma_28.shape)
print("melanoma_28 shape:", melanoma_28.shape)
print("melanocytic_nevi_28 shape:", melanocytic_nevi_28.shape)
print("vascular_lesions_28 shape:", vascular_lesions_28.shape)

actinic_keratoses_intraepithelial_carcinoma_28 shape: (228, 28, 28, 3)
basal_cell_carcinoma_28 shape: (359, 28, 28, 3)
benign_keratosis_like_lesions_28 shape: (769, 28, 28, 3)
dermatofibroma_28 shape: (473, 28, 28, 3)
melanoma_28 shape: (779, 28, 28, 3)
melanocytic_nevi_28 shape: (1000, 28, 28, 3)
vascular_lesions_28 shape: (473, 28, 28, 3)


### Por Clase

##### 0: actinic keratoses and intraepithelial carcinoma


In [4]:
df_clase_0 = pd.DataFrame({
    'R': actinic_keratoses_intraepithelial_carcinoma_28[:, :, :, 0].flatten(),
    'G': actinic_keratoses_intraepithelial_carcinoma_28[:, :, :, 1].flatten(),
    'B': actinic_keratoses_intraepithelial_carcinoma_28[:, :, :, 2].flatten()
})

df_clase_0

,R,G,B
0,158,111,117
1,161,116,121
2,164,121,130
3,167,127,135
4,166,133,142
...,...,...,...
178747,204,144,143
178748,203,144,140
178749,207,149,145
178750,197,140,133


##### 1: basal cell carcinoma


In [5]:
df_clase_1 = pd.DataFrame({
    'R': basal_cell_carcinoma_28[:, :, :, 0].flatten(),
    'G': basal_cell_carcinoma_28[:, :, :, 1].flatten(),
    'B': basal_cell_carcinoma_28[:, :, :, 2].flatten()
})
df_clase_1

,R,G,B
0,227,205,218
1,233,211,224
2,236,216,227
3,237,217,228
4,239,219,228
...,...,...,...
281451,198,141,130
281452,213,157,144
281453,207,151,138
281454,200,144,131


##### 2: benign keraosis-like lesions


In [6]:
df_clase_2 = pd.DataFrame({
    'R': benign_keratosis_like_lesions_28[:, :, :, 0].flatten(),
    'G': benign_keratosis_like_lesions_28[:, :, :, 1].flatten(),
    'B': benign_keratosis_like_lesions_28[:, :, :, 2].flatten()
})

df_clase_2

,R,G,B
0,196,170,169
1,195,169,168
2,195,169,172
3,197,170,175
4,201,174,181
...,...,...,...
602891,137,124,133
602892,129,116,125
602893,123,110,119
602894,94,81,90


##### 3: dermatofibroma


In [7]:
df_clase_3 = pd.DataFrame({
    'R': dermatofibroma_28[:, :, :, 0].flatten(),
    'G': dermatofibroma_28[:, :, :, 1].flatten(),
    'B': dermatofibroma_28[:, :, :, 2].flatten()
})

df_clase_3

,R,G,B
0,136,96,120
1,143,103,127
2,150,110,134
3,153,113,137
4,156,116,140
...,...,...,...
370827,221,137,135
370828,224,139,137
370829,225,139,136
370830,224,136,134


##### 4: melanoma


In [8]:
df_clase_4 = pd.DataFrame({
    'R': melanoma_28[:, :, :, 0].flatten(),
    'G': melanoma_28[:, :, :, 1].flatten(),
    'B': melanoma_28[:, :, :, 2].flatten()
})

df_clase_4

,R,G,B
0,216,188,187
1,217,189,188
2,220,192,191
3,222,194,193
4,221,195,196
...,...,...,...
610731,210,178,183
610732,213,178,182
610733,214,179,183
610734,216,180,182


##### 5: melanocytic nevi


In [9]:
df_clase_5 = pd.DataFrame({
    'R': melanocytic_nevi_28[:, :, :, 0].flatten(),
    'G': melanocytic_nevi_28[:, :, :, 1].flatten(),
    'B': melanocytic_nevi_28[:, :, :, 2].flatten()
})

df_clase_5

,R,G,B
0,246,176,178
1,247,177,179
2,247,178,181
3,245,179,181
4,246,179,186
...,...,...,...
783995,211,191,202
783996,208,191,201
783997,209,192,202
783998,209,193,203


##### 6: vascular lesions

In [10]:
df_clase_6 = pd.DataFrame({
    'R': vascular_lesions_28[:, :, :, 0].flatten(),
    'G': vascular_lesions_28[:, :, :, 1].flatten(),
    'B': vascular_lesions_28[:, :, :, 2].flatten()
})

df_clase_6


,R,G,B
0,229,134,140
1,229,138,143
2,230,145,150
3,229,153,155
4,228,160,161
...,...,...,...
370827,190,174,203
370828,190,174,203
370829,190,172,201
370830,189,170,200


### Total

El cuadro se genera mediante la sumatoria de los canales por clase

In [14]:
df_clase_t_s = pd.DataFrame({
    'R': [np.sum(df_clase_0['R']), np.sum(df_clase_1['R']), np.sum(df_clase_2['R']), np.sum(df_clase_3['R']), np.sum(df_clase_4['R']), np.sum(df_clase_5['R']), np.sum(df_clase_6['R'])],
    'G': [np.sum(df_clase_0['G']), np.sum(df_clase_1['G']), np.sum(df_clase_2['G']), np.sum(df_clase_3['G']), np.sum(df_clase_4['G']), np.sum(df_clase_5['G']), np.sum(df_clase_6['G'])],
    'B': [np.sum(df_clase_0['B']), np.sum(df_clase_1['B']), np.sum(df_clase_2['B']), np.sum(df_clase_3['B']), np.sum(df_clase_4['B']), np.sum(df_clase_5['B']), np.sum(df_clase_6['B'])]
})

df_clase_t_s.index = ['Clase 0', 'Clase 1', 'Clase 2', 'Clase 3', 'Clase 4', 'Clase 5', 'Clase 6']

df_clase_t_s.loc['Total Variable'] = df_clase_t_s.sum(axis=0)
df_clase_t_s['Total Clase'] = df_clase_t_s.sum(axis=1)


df_clase_t_s



,R,G,B,Total Clase
Clase 0,35801135,26763904,27863032,90428071
Clase 1,55741738,44073624,46047438,145862800
Clase 2,108751471,84414714,88770694,281936879
Clase 3,73938291,55048468,58400223,187386982
Clase 4,108908286,83344802,87230201,279483289
Clase 5,150635566,105226257,109472984,365334807
Clase 6,73368958,54215674,59952393,187537025
Total Variable,607145445,453087443,477736965,1537969853


El cuadro se genera mediante el promedio de los canales por clase

In [15]:
df_clase_t_p = pd.DataFrame({
    'R': [np.mean(df_clase_0['R']), np.mean(df_clase_1['R']), np.mean(df_clase_2['R']), np.mean(df_clase_3['R']), np.mean(df_clase_4['R']), np.mean(df_clase_5['R']), np.mean(df_clase_6['R'])],
    'G': [np.mean(df_clase_0['G']), np.mean(df_clase_1['G']), np.mean(df_clase_2['G']), np.mean(df_clase_3['G']), np.mean(df_clase_4['G']), np.mean(df_clase_5['G']), np.mean(df_clase_6['G'])],
    'B': [np.mean(df_clase_0['B']), np.mean(df_clase_1['B']), np.mean(df_clase_2['B']), np.mean(df_clase_3['B']), np.mean(df_clase_4['B']), np.mean(df_clase_5['B']), np.mean(df_clase_6['B'])]
})

df_clase_t_p.index = ['Clase 0', 'Clase 1', 'Clase 2', 'Clase 3', 'Clase 4', 'Clase 5', 'Clase 6']

df_clase_t_p = df_clase_t_p.round(0)

df_clase_t_p.loc['Total Variable'] = df_clase_t_p.sum(axis=0)
df_clase_t_p['Total Clase'] = df_clase_t_p.sum(axis=1)



df_clase_t_p

,R,G,B,Total Clase
Clase 0,200.0,150.0,156.0,506.0
Clase 1,198.0,157.0,164.0,519.0
Clase 2,180.0,140.0,147.0,467.0
Clase 3,199.0,148.0,157.0,504.0
Clase 4,178.0,136.0,143.0,457.0
Clase 5,192.0,134.0,140.0,466.0
Clase 6,198.0,146.0,162.0,506.0
Total Variable,1345.0,1011.0,1069.0,3425.0


#### Funciones

In [60]:
def grados_libertad(df):
    c = len(df) - 1
    r = len(df.columns) - 1
    return (r - 1) * (c - 1)

def chiCuadrada(df):
    n = df.iloc[-1, -1]
    errores = []
    esperados = []
    E = []

    for i in range(0, len(df) - 1):
        for j in range(0, len(df.columns) - 1):
            p = (df.iloc[i, -1]/n) * (df.iloc[-1, j]/n)
            E.append(p * n)
        esperados.append(E.copy())
        E.clear()

    esperados = np.round(np.array(esperados), 4)

    for i in range(0, len(df) - 1):
        for j in range(0, len(df.columns) - 1):
            temp = (df.iloc[i,j] - esperados[i,j])**2 / esperados[i,j]
            errores.append(temp)

    errores = np.round(np.array(errores), 4)
    suma = np.sum(errores).round(4)

    return esperados, errores, suma

def evaluar_p(p):
    if p > 0.05:
        print('\nSe rechaza la hipotesis nula.\nLas variables no son independientes')
    else:
        print('\nSe acepta la hipotesis nula.\nLas variables son independientes')


#### Obtencion de Chicuadrada para cada tabla

Se realiza el calculo de los grados de libertad, de los valores esperados de los errores el p-valor y la evaluación en torno a un alpha de 0.05 tanto para la tbla hecha con las sumatorias de los canales como de la tabla hecha por el promedio por color de cada clase

Tabla de la sumatoria

In [61]:
gl_s = grados_libertad(df_clase_t_s)
x_2_s = chiCuadrada(df_clase_t_s)
p_v_s = chi2.sf(x_2_s[2], gl_s)

print(f'\nGrados de libertad: {gl_s}')
print(f'\nValores Esperados:\n{x_2_s[0]}')
print(f'\nErrores MSE:\n{x_2_s[1]}')
print(f'\nValor de ChiCuadrada: {x_2_s[2]}')
print(f'\np-valor resultante: {p_v_s}')
evaluar_p(p_v_s)



Grados de libertad: 12

Valores Esperados:
[[3.56983535e+07 2.66401993e+07 2.80895182e+07]
 [5.75823606e+07 4.29713254e+07 4.53091140e+07]
 [1.11300421e+08 8.30588840e+07 8.75775742e+07]
 [7.39748912e+07 5.52043906e+07 5.82077002e+07]
 [1.10331815e+08 8.23360539e+07 8.68154197e+07]
 [1.44223480e+08 1.07627996e+08 1.13483331e+08]
 [7.40341238e+07 5.52485934e+07 5.82543078e+07]]

Errores MSE:
[2.95925300e+02 5.74426700e+02 1.82616170e+03 5.88355774e+04
 2.82761141e+04 1.20311853e+04 5.83748488e+04 2.21321903e+04
 1.62545595e+04 1.81085000e+01 4.40397300e+02 6.36772200e+02
 1.83667410e+04 1.23587736e+04 1.98171640e+03 2.85077367e+05
 5.35952683e+04 1.41720224e+05 5.97623800e+03 1.93113072e+04
 4.94983732e+04]

Valor de ChiCuadrada: 787582.2763

p-valor resultante: 0.0

Se acepta la hipotesis nula.
Las variables son independientes


Tabla de los promedios

In [62]:
gl_p = grados_libertad(df_clase_t_p)
x_2_p = chiCuadrada(df_clase_t_p)
p_v_p = chi2.sf(x_2_p[2], gl_p)

print(f'\nGrados de libertad: {gl_p}')
print(f'\nValores Esperados:\n{x_2_p[0]}')
print(f'\nErrores MSE:\n{x_2_p[1]}')
print(f'\nValor de ChiCuadrada: {x_2_p[2]}')
print(f'\np-valor resultante: {p_v_p}')
evaluar_p(p_v_p)


Grados de libertad: 12

Valores Esperados:
[[198.7066 149.3623 157.9311]
 [203.8117 153.1997 161.9886]
 [183.3912 137.8502 145.7585]
 [197.9212 148.772  157.3069]
 [179.4642 134.8984 142.6374]
 [182.9985 137.555  145.4464]
 [198.7066 149.3623 157.9311]]

Errores MSE:
[0.0084 0.0027 0.0236 0.1657 0.0943 0.025  0.0627 0.0335 0.0106 0.0059
 0.004  0.0006 0.0119 0.009  0.0009 0.4428 0.0919 0.2039 0.0025 0.0757
 0.1048]

Valor de ChiCuadrada: 1.3804

p-valor resultante: 0.9999165874631681

Se rechaza la hipotesis nula.
Las variables no son independientes


Dependiendo de el tipo de tabla que decidamos usar el resultado variara en mayor o menor medida, por un lado con ambas tablas contando con 12 grados de libertad vemos que el p-valor de la tabla de sumatoria es 0.0 mientras que la de promedios es 0.9999